In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.GradientDescentLS(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.2528630793094635
epoch:  1, loss: 0.15897618234157562
epoch:  2, loss: 0.10568813979625702
epoch:  3, loss: 0.07483512908220291
epoch:  4, loss: 0.056644998490810394
epoch:  5, loss: 0.045877814292907715
epoch:  6, loss: 0.03950885683298111
epoch:  7, loss: 0.03574659675359726
epoch:  8, loss: 0.03352739289402962
epoch:  9, loss: 0.032219771295785904
epoch:  10, loss: 0.03145007789134979
epoch:  11, loss: 0.0309968963265419
epoch:  12, loss: 0.030729608610272408
epoch:  13, loss: 0.030571434646844864
epoch:  14, loss: 0.030477138236165047
epoch:  15, loss: 0.03042026236653328
epoch:  16, loss: 0.03038528747856617
epoch:  17, loss: 0.03038029745221138
epoch:  18, loss: 0.030342860147356987
epoch:  19, loss: 0.03034265711903572
epoch:  20, loss: 0.030302628874778748
epoch:  21, loss: 0.030277609825134277
epoch:  22, loss: 0.03026353381574154
epoch:  23, loss: 0.030236603692173958
epoch:  24, loss: 0.030224870890378952
epoch:  25, loss: 0.030195973813533783
epoch:  26, 

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7819220835915011
Test metrics:  R2 = 0.8194488293559897
